In [1]:
import pandas as pd

In [2]:
hero_df = pd.read_csv("Resources/purchase_data.csv")
hero_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Number of unique players

unique_players_df = hero_df.drop_duplicates(subset = ["SN"])

unique = len(unique_players_df["SN"])

pd.DataFrame({"Total Players": [unique]})

,Total Players
0,576


In [4]:
# Number of unique items, average price, number of purchases, and total revenue
# Money formatted with $ and two decimals

pd.DataFrame({
    "Number of Unique Items": [len(hero_df['Item Name'].unique())],
    "Average Price": ['${:.2f}'.format(hero_df['Price'].mean())],
    "Number of Purchases": [hero_df['Item Name'].count()],
    "Total Revenue": ['${:,.2f}'.format(hero_df['Price'].sum())]
})

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
# Gender Demographics

males = unique_players_df.groupby("Gender")["SN"].count().loc["Male"]
females = unique_players_df.groupby("Gender")["SN"].count().loc["Female"]
others = unique_players_df.groupby("Gender")["SN"].count().loc["Other / Non-Disclosed"]



demographics_df = pd.DataFrame({
    "Total Count": [males, females, others],
    "Percentage of Players": ['{:.2f}%'.format(males * 100 / unique),
                              '{:.2f}%'.format(females * 100 / unique),
                              '{:.2f}%'.format(others * 100 / unique)]
    }).rename(index = {0: "Male", 1: "Female", 2: "Other / Non-Disclosed"})

demographics_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [6]:
# Purchasing Analysis (Gender)

gp_df = hero_df[["Gender", "Price"]]
pa_gender_df = pd.DataFrame({
    "Purchase Count": gp_df.groupby("Gender")["Price"].count(),
    "Total Purchase Value": gp_df.groupby("Gender")["Price"].sum()
})
pa_gender_df["Average Purchase Price"] = (pa_gender_df["Total Purchase Value"] / pa_gender_df["Purchase Count"]).map("${:.2f}".format)
pa_gender_df["Avg Total Purchase per Person"] = (pa_gender_df["Total Purchase Value"] / demographics_df["Total Count"].sort_index(ascending = True)).map("${:.2f}".format)
pa_gender_df["Total Purchase Value"] = pa_gender_df["Total Purchase Value"].map("${:.2f}".format)

pa_gender_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
# Age Demographics

ages_df = unique_players_df[["SN", "Age", "Price"]].copy()
# print(f'min:{min(ages_df["Age"])} max:{max(ages_df["Age"])}')
# min 7 max 45

bins = [0, 9, 14, 19, 24, 29, 34, 39, 45]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
ages_df["Bins"] = pd.cut(ages_df["Age"], bins, labels = group_names, include_lowest = True)

age_demographics_df = pd.DataFrame({
    "Total Count": ages_df.groupby("Bins")["Age"].count(),
    "Percentage of Players": (ages_df.groupby("Bins")["Age"].count() * 100 / len(ages_df)).map("{:.2f}%".format)
}, index = group_names)

age_demographics_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%
